sql로 EDA

CSV -> db 변환 (sql 사용을 위함)

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# CSV 경로 (EDA.ipynb 기준 상대경로)
csv_path = "../datasets/PaySim.csv"

# CSV 로드
df = pd.read_csv(csv_path)

# SQLite DB 생성/연결 (local DB 파일 생성됨)
engine = create_engine("sqlite:///paysim.db")

# PaySim이라는 테이블명으로 저장
df.to_sql("PaySim", engine, if_exists="replace", index=False)

print("테이블 생성 완료!")


테이블 생성 완료!


In [1]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%sql sqlite:///paysim.db

---

### **✔️  1. 기본 구조 확인**

1) 컬럼 정보 확인

In [22]:
%%sql
PRAGMA table_info(PaySim);

 * sqlite:///paysim.db
   sqlite:///test.db
Done.


cid,name,type,notnull,dflt_value,pk
0,step,BIGINT,0,None,0
1,type,TEXT,0,None,0
2,amount,FLOAT,0,None,0
3,nameOrig,TEXT,0,None,0
4,oldbalanceOrg,FLOAT,0,None,0
5,newbalanceOrig,FLOAT,0,None,0
6,nameDest,TEXT,0,None,0
7,oldbalanceDest,FLOAT,0,None,0
8,newbalanceDest,FLOAT,0,None,0
9,isFraud,BIGINT,0,None,0


2) 전체 행 개수 확인

In [28]:
%%sql
SELECT COUNT(*) AS total_rows
FROM PaySim;



 * sqlite:///paysim.db
   sqlite:///test.db
Done.


total_rows
114957


3) 숫자형 컬럼들의 기본 통계 요약

In [30]:
%%sql
SELECT
    MIN(amount) AS min_amount,
    MAX(amount) AS max_amount,
    AVG(amount) AS avg_amount,
    SUM(amount) AS total_amount,
    MIN(oldbalanceOrg) AS min_oldOrg,
    MAX(oldbalanceOrg) AS max_oldOrg,
    AVG(oldbalanceOrg) AS avg_oldOrg
FROM PaySim;



 * sqlite:///paysim.db
   sqlite:///test.db
Done.


min_amount,max_amount,avg_amount,total_amount,min_oldOrg,max_oldOrg,avg_oldOrg
0.92,36973901.85,180774.60720133607,20781306520.04399,0.0,33593209.9,844075.735727881


4) 고유값 조회

In [27]:
%%sql
SELECT DISTINCT type FROM PaySim;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


type
CASH_IN
PAYMENT
TRANSFER
CASH_OUT
DEBIT


---

### **✔️  2. 결측치 / 데이터 품질 점검**

5) 컬럼별 결측치 개수

In [31]:
%%sql
SELECT
    SUM(CASE WHEN step IS NULL THEN 1 ELSE 0 END) AS step_null,
    SUM(CASE WHEN type IS NULL THEN 1 ELSE 0 END) AS type_null,
    SUM(CASE WHEN amount IS NULL THEN 1 ELSE 0 END) AS amount_null,
    SUM(CASE WHEN nameOrig IS NULL THEN 1 ELSE 0 END) AS nameOrig_null,
    SUM(CASE WHEN oldbalanceOrg IS NULL THEN 1 ELSE 0 END) AS oldbalanceOrg_null,
    SUM(CASE WHEN newbalanceOrig IS NULL THEN 1 ELSE 0 END) AS newbalanceOrig_null,
    SUM(CASE WHEN nameDest IS NULL THEN 1 ELSE 0 END) AS nameDest_null,
    SUM(CASE WHEN oldbalanceDest IS NULL THEN 1 ELSE 0 END) AS oldbalanceDest_null,
    SUM(CASE WHEN newbalanceDest IS NULL THEN 1 ELSE 0 END) AS newbalanceDest_null,
    SUM(CASE WHEN isFraud IS NULL THEN 1 ELSE 0 END) AS isFraud_null,
    SUM(CASE WHEN isFlaggedFraud IS NULL THEN 1 ELSE 0 END) AS isFlaggedFraud_null,
    SUM(CASE WHEN signup_step IS NULL THEN 1 ELSE 0 END) AS signup_step_null,
    SUM(CASE WHEN isActivation IS NULL THEN 1 ELSE 0 END) AS isActivation_null,
    SUM(CASE WHEN isReferral IS NULL THEN 1 ELSE 0 END) AS isReferral_null
FROM PaySim;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


step_null,type_null,amount_null,nameOrig_null,oldbalanceOrg_null,newbalanceOrig_null,nameDest_null,oldbalanceDest_null,newbalanceDest_null,isFraud_null,isFlaggedFraud_null,signup_step_null,isActivation_null,isReferral_null
0,0,0,0,0,0,0,0,0,0,0,0,0,0


5) 중복 행 개수

SQLite에서는 아래처럼 전체 행 중복을 직접 비교하는 기능이 없음
-> 문자열을 이어 붙이는(concatenation) 연산자를 사용해 여러 컬럼을 하나로 붙여서 각 row를 하나의 문자열 key처럼 취급
-> 중복된 문자열 key 개수를 세는 방식

In [32]:
%%sql
SELECT COUNT(*) - COUNT(DISTINCT step || type || amount || nameOrig || nameDest) 
AS approx_duplicate_rows
FROM PaySim;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


approx_duplicate_rows
0


---

### **✔️  3. Fraud 데이터 분석에 필요한 기본 숫자/관계**

7) Fraud 비율

In [33]:
%%sql
SELECT
    isFraud,
    COUNT(*) AS cnt,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM PaySim), 2) AS ratio
FROM PaySim
GROUP BY isFraud;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


isFraud,cnt,ratio
0,114786,99.85
1,171,0.15


8) Flagged Fraud 비율

In [2]:
%%sql
SELECT
    isFlaggedFraud,
    COUNT(*) AS cnt
FROM PaySim
GROUP BY isFlaggedFraud;


 * sqlite:///paysim.db
Done.


isFlaggedFraud,cnt
0,114956
1,1


9) Fraud 트랜잭션에서 amount 평균/최대/합계

In [3]:
%%sql
SELECT
    AVG(amount) AS avg_fraud_amount,
    MAX(amount) AS max_fraud_amount,
    SUM(amount) AS total_fraud_amount
FROM PaySim
WHERE isFraud = 1;


 * sqlite:///paysim.db
Done.


avg_fraud_amount,max_fraud_amount,total_fraud_amount
1272081.2662097088,10000000.0,217525896.52186018


10) Fraud 거래의 잔액 변화 특징

In [5]:
%%sql
SELECT
    AVG(oldbalanceOrg) AS avg_oldbalance,
    AVG(newbalanceOrig) AS avg_newbalance,
    AVG(oldbalanceDest) AS avg_dest_old,
    AVG(newbalanceDest) AS avg_dest_new
FROM PaySim
WHERE isFraud = 1;


 * sqlite:///paysim.db
Done.


avg_oldbalance,avg_newbalance,avg_dest_old,avg_dest_new
1309573.048128655,56360.409999999996,267416.90771929827,815817.2914619884


---

### **✔️  4. 잔액 관련 관계 분석**

11) 송금자(oldbalanceOrg) - 거래금액(amount) 관계

In [10]:
%%sql
SELECT
    ROUND(oldbalanceOrg / 1000000) AS org_balance_bin, 
    COUNT(*) AS cnt,
    AVG(amount) AS avg_amount
FROM PaySim
GROUP BY org_balance_bin
ORDER BY org_balance_bin;


 * sqlite:///paysim.db
Done.


org_balance_bin,cnt,avg_amount
0.0,98728,182256.34169140868
1.0,3945,166079.91113744225
2.0,1951,174606.95399281668
3.0,1707,176156.35622487628
4.0,1332,173880.14397377888
5.0,1209,192990.5712991777
6.0,977,162948.22530403858
7.0,842,171712.72667887845
8.0,764,161242.49720117118
9.0,606,189790.1767834765


12) 수취인(oldbalanceDest) - 거래금액 관계

In [12]:
%%sql
SELECT
    ROUND(oldbalanceDest / 1000000) AS dest_balance_bin,
    COUNT(*) AS cnt,
    AVG(amount) AS avg_amount
FROM PaySim
GROUP BY dest_balance_bin
ORDER BY dest_balance_bin;


 * sqlite:///paysim.db
Done.


dest_balance_bin,cnt,avg_amount
0.0,74873,108186.41181045918
1.0,18948,230910.3272155182
2.0,7479,284278.13854199945
3.0,4155,315608.93009806756
4.0,2475,328655.8982189881
5.0,1733,381559.31919021177
6.0,1122,370542.0575747874
7.0,893,385924.0350226364
8.0,657,414231.65077167144
9.0,471,443922.90654816426


---

### **✔️  5. 사용자 행동/계정 관련(signup_step, isActivation, isReferral)**

13) 가입 시점(signup_step)별 활성화 여부

In [14]:
%%sql
SELECT signup_step, isActivation, COUNT(*) AS cnt
FROM PaySim
GROUP BY signup_step, isActivation
limit 20;


 * sqlite:///paysim.db
Done.


signup_step,isActivation,cnt
1,0,22
1,1,26
2,0,4
2,1,10
3,0,6
3,1,2
4,0,9
4,1,6
5,0,12
5,1,5


14) 추천 사용자(isReferral) 비율

In [15]:
%%sql
SELECT
    isReferral,
    COUNT(*) AS cnt
FROM PaySim
GROUP BY isReferral;


 * sqlite:///paysim.db
Done.


isReferral,cnt
0,109179
1,5778


15) Referral 유저의 Fraud 비율

In [16]:
%%sql
SELECT
    isReferral,
    SUM(isFraud) AS fraud_count,
    COUNT(*) AS total,
    ROUND(SUM(isFraud) * 100.0 / COUNT(*), 2) AS fraud_ratio
FROM PaySim
GROUP BY isReferral;


 * sqlite:///paysim.db
Done.


isReferral,fraud_count,total,fraud_ratio
0,163,109179,0.15
1,8,5778,0.14


---

### **✔️  6. 트랜잭션 흐름/step 기반 분석**

16) step별 거래 건수

In [25]:
%%sql
SELECT step, COUNT(*) AS cnt
FROM PaySim
GROUP BY step
ORDER BY cnt
limit 20;


 * sqlite:///paysim.db
Done.


step,cnt
27,1
28,1
29,1
51,1
52,1
76,1
77,1
100,1
101,1
123,1


17) 시간(step)별 Fraud 발생량

In [23]:
%%sql
SELECT step, SUM(isFraud) AS fraud_count
FROM PaySim
GROUP BY step
ORDER BY step
limit 20;


 * sqlite:///paysim.db
Done.


step,fraud_count
1,0
2,0
3,0
4,0
5,1
6,0
7,0
8,0
9,1
10,0


---

### **✔️  7. 계좌명(nameOrig, nameDest) 분석**

18) 가장 많이 거래한 Origin ID

In [24]:
%%sql
SELECT nameOrig, COUNT(*) AS cnt
FROM PaySim
GROUP BY nameOrig
ORDER BY cnt DESC
LIMIT 20;


 * sqlite:///paysim.db
Done.


nameOrig,cnt
C999898340,5
C999476777,5
C997684410,5
C996650520,5
C996577814,5
C995924343,5
C995831718,5
C9950119,5
C993445479,5
C991033178,5


19) 목적지(nameDest) 상위 20

In [26]:
%%sql
SELECT nameDest, COUNT(*) AS cnt
FROM PaySim
GROUP BY nameDest
ORDER BY cnt DESC
LIMIT 20;


 * sqlite:///paysim.db
Done.


nameDest,cnt
C756334542,11
C530706181,10
C2036393827,10
C1568538856,10
C681078805,9
C220294170,9
C16880623,9
C702393884,8
C626288798,8
C357066329,8


---

### **✔️  8. 잔액 불일치 체크 (이상치 탐색)**

20) oldbalanceOrg + amount ≠ newbalanceOrg

In [30]:
%%sql
SELECT *
FROM PaySim
WHERE ABS((oldbalanceOrg - amount) - newbalanceOrig) > 0.01
LIMIT 50;


 * sqlite:///paysim.db
Done.


step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,signup_step,isActivation,isReferral
278,CASH_IN,330218.42,C632336343,20866.0,351084.42,C834976624,452419.57,122201.15,0,0,278,0,0
10,CASH_IN,152264.21,C1746846248,106589.0,258853.21,C1607284477,201303.01,49038.8,0,0,10,0,0
403,TRANSFER,1551760.63,C333676753,0.0,0.0,C1564353608,3198359.45,4750120.08,0,0,403,1,0
206,CASH_IN,78172.3,C813403091,2921331.58,2999503.88,C1091768874,415821.9,337649.6,0,0,206,0,0
259,PAYMENT,915.13,C2002954533,0.0,0.0,M290849763,0.0,0.0,0,0,259,1,0
188,CASH_OUT,20603.87,C813757373,0.0,0.0,C823291717,558068.66,578672.53,0,0,188,0,0
139,CASH_OUT,58605.72,C1850864812,0.0,0.0,C618657299,585494.94,644100.66,0,0,139,0,0
230,PAYMENT,4865.11,C886849972,0.0,0.0,M623175144,0.0,0.0,0,0,230,1,0
544,CASH_OUT,118131.63,C390714641,0.0,0.0,C366360355,8131691.35,8476246.86,0,0,544,0,0
45,CASH_OUT,141100.88,C1514989792,80506.0,0.0,C409578677,89384.09,230484.96,0,0,45,0,0


21) oldbalanceDest + amount ≠ newbalanceDest

In [31]:
%%sql
SELECT *
FROM PaySim
WHERE ABS((oldbalanceDest + amount) - newbalanceDest) > 0.01
LIMIT 50;


 * sqlite:///paysim.db
Done.


step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,signup_step,isActivation,isReferral
278,CASH_IN,330218.42,C632336343,20866.0,351084.42,C834976624,452419.57,122201.15,0,0,278,0,0
15,PAYMENT,11647.08,C1264712553,30370.0,18722.92,M215391829,0.0,0.0,0,0,15,1,0
10,CASH_IN,152264.21,C1746846248,106589.0,258853.21,C1607284477,201303.01,49038.8,0,0,10,0,0
206,CASH_IN,78172.3,C813403091,2921331.58,2999503.88,C1091768874,415821.9,337649.6,0,0,206,0,0
259,PAYMENT,915.13,C2002954533,0.0,0.0,M290849763,0.0,0.0,0,0,259,1,0
230,PAYMENT,4865.11,C886849972,0.0,0.0,M623175144,0.0,0.0,0,0,230,1,0
544,CASH_OUT,118131.63,C390714641,0.0,0.0,C366360355,8131691.35,8476246.86,0,0,544,0,0
45,CASH_OUT,141100.88,C1514989792,80506.0,0.0,C409578677,89384.09,230484.96,0,0,45,0,0
211,CASH_OUT,153486.36,C513018655,20476.0,0.0,C1302281681,85173.16,238659.53,0,0,211,0,0
302,CASH_OUT,51042.59,C122114408,184.0,0.0,C705120650,380143.14,431185.74,0,0,302,0,0


---

### **✔️  9. 고위험 패턴 탐색**

22) 잔액이 없는데 큰 금액 거래

In [33]:
%%sql
SELECT *
FROM PaySim
WHERE oldbalanceOrg = 0 AND amount > 10000
limit 20;


 * sqlite:///paysim.db
Done.


step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,signup_step,isActivation,isReferral
403,TRANSFER,1551760.63,C333676753,0.0,0.0,C1564353608,3198359.45,4750120.08,0,0,403,1,0
188,CASH_OUT,20603.87,C813757373,0.0,0.0,C823291717,558068.66,578672.53,0,0,188,0,0
139,CASH_OUT,58605.72,C1850864812,0.0,0.0,C618657299,585494.94,644100.66,0,0,139,0,0
544,CASH_OUT,118131.63,C390714641,0.0,0.0,C366360355,8131691.35,8476246.86,0,0,544,0,0
18,PAYMENT,48691.16,C1810046508,0.0,0.0,M487081598,0.0,0.0,0,0,18,1,0
132,CASH_OUT,240156.34,C807101143,0.0,0.0,C1531623971,2811902.28,3052058.62,0,0,132,0,0
283,CASH_OUT,44360.26,C346953986,0.0,0.0,C526046232,282664.52,327024.78,0,0,283,0,0
326,PAYMENT,26565.95,C1512675672,0.0,0.0,M1330826048,0.0,0.0,0,0,326,1,1
208,PAYMENT,24649.07,C1353964860,0.0,0.0,M1059671726,0.0,0.0,0,0,208,1,0
226,CASH_OUT,176016.04,C800760258,0.0,0.0,C358043414,12916654.7,13005481.5,0,0,226,0,0


23) isActivation 후 즉시 Fraud

In [35]:
%%sql
SELECT *
FROM PaySim
WHERE isActivation = 1 AND isFraud = 1
limit 20;


 * sqlite:///paysim.db
Done.


step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,signup_step,isActivation,isReferral
212,TRANSFER,4953893.08,C728984460,4953893.08,4953893.08,C639921569,0.0,0.0,1,1,212,1,0
223,TRANSFER,334890.91,C1711836423,334890.91,0.0,C1676868220,0.0,0.0,1,0,223,1,1
621,TRANSFER,2060748.45,C1129518026,2060748.45,0.0,C1288050386,0.0,0.0,1,0,621,1,0
194,TRANSFER,1501297.88,C1043170433,1501297.88,0.0,C768896772,0.0,0.0,1,0,194,1,0
672,TRANSFER,330756.72,C2036720607,330756.72,0.0,C2048695496,0.0,0.0,1,0,672,1,0
125,TRANSFER,621569.61,C288917283,621569.61,0.0,C652970534,0.0,0.0,1,0,125,1,0
684,TRANSFER,537025.01,C1744276151,537025.01,0.0,C38221870,0.0,0.0,1,0,684,1,0
659,TRANSFER,50577.26,C1316214248,50577.26,0.0,C258297830,0.0,0.0,1,0,659,1,0
613,TRANSFER,1892137.66,C1387416875,1892137.66,0.0,C1999528314,0.0,0.0,1,0,613,1,0
413,TRANSFER,115085.31,C2039186549,115085.31,0.0,C544794295,0.0,0.0,1,0,413,1,0


---

### **✔️  10. 전체 요약 대시보드 수준 (핵심 수치 한 번에)**

24) 전체 요약(코어 메트릭)

In [36]:
%%sql
SELECT
    COUNT(*) AS total_rows,
    SUM(isFraud) AS fraud_rows,
    SUM(isFlaggedFraud) AS flagged_rows,
    AVG(amount) AS avg_amount,
    MAX(amount) AS max_amount,
    AVG(oldbalanceOrg) AS avg_old_org,
    AVG(oldbalanceDest) AS avg_old_dest
FROM PaySim;


 * sqlite:///paysim.db
Done.


total_rows,fraud_rows,flagged_rows,avg_amount,max_amount,avg_old_org,avg_old_dest
114957,171,1,180774.60720133607,36973901.85,844075.735727881,1102911.619571927


---